In [53]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import pandas as pd
import os

In [54]:

# Hyperparameters
lookback_window = 7  # Number of time steps
#I don't have any idea which number is best
# from past test it looks like model has an understanding of general trend of the stock
# but not the specific chracteristics of the stock
# chatgpt recommends me to increase lookback window
input_size = 6        # Number of features (univariate = 1)
hidden_size = 50      # Number of LSTM neurons
num_layers = 2        # Number of LSTM layers
output_size = 1       # Predicting one value
learning_rate = 0.001
epochs = 50
batch_size = 32


In [55]:
dataLocation="../Data/StandardizedData"
allcsvs=os.listdir(dataLocation)
# turn float64 to float 32

In [56]:
unnecessaryColumns=["Date","Pct_Change"]

# from Scaler import Scaler

# scaler = Scaler()


def df_to_X_y(df,window_size=5):
  lightWeightDf=df.drop(columns=unnecessaryColumns)
  changeMatrix=lightWeightDf.copy()
  changeMatrix["Open"]=changeMatrix["Open"].pct_change()
  changeMatrix["High"]=changeMatrix["High"].pct_change()
  changeMatrix["Low"]=changeMatrix["Low"].pct_change()
  changeMatrix["Close"]=changeMatrix["Close"].pct_change()
  changeMatrix["Volume"]=changeMatrix["Volume"].pct_change()
  changeMatrix.dropna(inplace=True)
  changeMatrix.fillna(0,inplace=True)
  # df_as_np = scaler.fit_transform(lightWeightDf.to_numpy())
  df_as_np = changeMatrix.to_numpy()
  X = []
  y = []
  for i in range(len(df_as_np)-window_size):
    row = [a for a in df_as_np[i:i+window_size]]
    X.append(row)
    label = df_as_np[i+window_size,3]
    y.append(label)
  return np.array(X), np.array(y)

In [57]:
XAll = np.zeros(1, dtype=np.int32)
yAll = np.zeros(1, dtype=np.int32)

for i in range(len(allcsvs)):
    rawData=pd.read_csv(os.path.join(dataLocation,allcsvs[i]))
    Xtemp,yTemp=df_to_X_y(rawData ,lookback_window)
    if(XAll.shape[0] == 1 and yAll.shape[0] == 1):
        XAll=Xtemp
        yAll=yTemp
    else:
        XAll=np.concatenate((XAll,Xtemp),axis=0)
        yAll=np.concatenate((yAll,yTemp),axis=0)


# I have to store all of the rawData accurately because I need them afterwards
# or I can make a huge tensor and store all of them in it
# but this raises concerns about the time series consistency

In [58]:
XAll.shape, yAll.shape

((57412, 7, 6), (57412,))

In [60]:
XAll[0]

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00],
       [ 5.00270958e-04, -1.99198488e-03, -1.51215973e-03,
         2.51626391e-03, -3.47952449e-02,  1.00000000e+00],
       [-5.00086055e-04,  7.98403856e-03,  7.06716900e-03,
         9.03612800e-03, -9.78712104e-02,  1.00000000e+00],
       [ 7.50373849e-03,  4.95037801e-03,  3.50878731e-03,
         2.98506920e-03,  9.01759592e-02,  1.00000000e+00],
       [-9.92830937e-04, -4.92586393e-03, -1.09889798e-02,
        -1.53768484e-02,  3.07745209e-02,  3.00000000e+00],
       [-1.34195861e-02, -1.73268699e-02, -2.22221915e-02,
        -2.21663132e-02, -7.50415494e-02,  2.00000000e+00],
       [-2.01511832e-02, -1.51135189e-02, -1.44629396e-02,
        -1.23648044e-02,  4.88214480e-02,  1.00000000e+00]])

In [59]:
assert False,"stop"

AssertionError: stop

In [45]:
def shuffle_data(X, y):
    """
    Shuffle the data while maintaining the correspondence between X and y.

    Parameters:
    X (numpy.ndarray): The input features tensor.
    y (numpy.ndarray): The target array.

    Returns:
    tuple: Shuffled X and y tensors.
    """
    assert len(X) == len(y), "X and y must have the same length."
    
    # Generate a random permutation of indices
    #TODO add a seed for reproducibility
    #add seed to the code below
    np.random.seed(42)
    indices = np.random.permutation(len(X))
    # print(indices[:5])
    # Shuffle X and y
    X_shuffled = X[indices]
    y_shuffled = y[indices]
    
    return X_shuffled, y_shuffled

In [46]:
X_shuffled, y_shuffled = shuffle_data(XAll, yAll)

In [47]:
class TimeSeriesDataset(Dataset):
    def __init__(self, X_shuffled, y_shuffled):
        # self.scaler = Scaler()
        # X = np.zeros(1)
        # y = np.zeros(1)

        # for i in range(len(allcsvLoc)):
        #     rawData=pd.read_csv(os.path.join(dataDir,allcsvLoc[i]))
        #     Xtemp,yTemp=df_to_X_y(rawData, self.scaler, lookback_window)
        #     if(X.shape[0] == 1 and y.shape[0] == 1):
        #         X=Xtemp
        #         y=yTemp
        #     else:
        #         X=np.concatenate((X,Xtemp),axis=0)
        #         y=np.concatenate((y,yTemp),axis=0)


        # #shuffle the data
        # #NOTE shuffling is automatically done in the dataloader
        # X_shuffled, y_shuffled = shuffle_data(X, y)
        self.dataX = torch.tensor(X_shuffled, dtype=torch.float32)
        self.dataY = torch.tensor(y_shuffled, dtype=torch.float32)
        
        

    def __len__(self):
        return self.dataX.size(0)

    def __getitem__(self, idx):
        sequence = self.dataX[idx, :]
        target = self.dataY[idx]
        return sequence, target

# LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        output, _ = self.lstm(x)
        last_output = output[:, -1, :]  # Take the output at the last time step
        return self.fc(last_output)
    
    # def init_weights(module):
    #     if isinstance(module, nn.LSTM):
    #         nn.init.xavier_normal_(module.weight_ih_l0)
    #         nn.init.xavier_normal_(module.weight_hh_l0)
    #         if module.is_bidirectional:
    #             nn.init.xavier_normal_(module.weight_ih_l0_reverse)
    #             nn.init.xavier_normal_(module.weight_hh_l0_reverse) 


In [48]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"Using {device}")

Using cuda


In [49]:
X_train = X_shuffled[:int(0.8*len(X_shuffled))]
y_train = y_shuffled[:int(0.8*len(y_shuffled))]
X_val = X_shuffled[int(0.8*len(X_shuffled)):]
y_val = y_shuffled[int(0.8*len(y_shuffled)):]

In [ ]:
trainDataset = TimeSeriesDataset(X_train, y_train)
train_loader = DataLoader(trainDataset, batch_size=batch_size, shuffle=True)
#Dataset for validation
valDataset = TimeSeriesDataset(X_val, y_val)
val_loader = DataLoader(valDataset, batch_size=batch_size, shuffle=False)

# Cross-validation: On non-overlapping sequential splits for evaluation.


# Initialize model, loss function, and optimizer
model = LSTMModel(input_size, hidden_size, num_layers, output_size)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(epochs):
    model.train()
    epoch_loss = 0

    for sequences, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(sequences)
        loss = criterion(outputs.squeeze(), targets)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss / len(train_loader):.4f}")

Epoch 1/50, Loss: nan
Epoch 2/50, Loss: nan
Epoch 3/50, Loss: nan


KeyboardInterrupt: 

In [ ]:
# Extract one batch of data for testing
for sequences, targets in train_loader:
    testInput = sequences  # Sequence data
    print(f"Test Input Shape: {testInput.shape}")
    realOutput = targets   # Ground truth output
    break

# Ensure the model is in evaluation mode
model.eval()

# Predict using the model
with torch.no_grad():  # Disable gradient calculations for inference
    predicted = model(testInput).squeeze()  # Forward pass through the model

# Check shapes and results
print(f"Predicted Output Shape: {predicted.shape}")
print(f"First Prediction: {predicted[0].item()}")
print(f"Actual Output (First Target): {realOutput[0].item()}")

predicted
# testInput.shape

Test Input Shape: torch.Size([32, 5, 6])
Predicted Output Shape: torch.Size([32])
First Prediction: 407.726318359375
Actual Output (First Target): 448.0087585449219


tensor([407.7263, 407.7263, 407.7263, 407.7263, 407.7263, 407.7263, 407.7263,
        407.7263, 407.7263, 407.7263, 407.7263, 407.7263, 407.7263, 407.7263,
        407.7263, 407.7263, 407.7263, 407.7263, 407.7263, 407.7263, 407.7263,
        407.7263, 407.7263, 407.7263, 407.7263, 407.7263, 407.7263, 407.7263,
        407.7263, 407.7263, 945.0376, 407.7263])

In [14]:
realOutput

tensor([ 448.0088,   25.0103,  168.0784,   20.9696,   15.5573, 1723.0000,
          30.5591,   33.7800,  172.0681,   22.8000,  103.7000,    4.6216,
          44.7600,  298.2500,    4.6500,   24.6301,   24.1800,    5.0271,
           8.8564,   47.9817,    3.2000,   46.2337,   20.6925,    7.3362,
         113.8000,   22.7703,   36.7012,   66.4421,    7.8999,    6.3868,
        1963.6000,  155.6090])